## Reading the pdf file

In [2]:
import PyPDF2

In [3]:
# read your file
file_path = './pdfs/FACTURE DECEMBRE 22 (1) - 10pages hidden (2) 10.pdf'
# file_path = './pdfs/test.pdf'
with open(file_path, 'rb') as f:
    pdf = PyPDF2.PdfFileReader(f)

    # get the first page
    page = pdf.getPage(0)

    # extract text from page
    text = page.extractText()

    # do whatever you want with extracted text
    print(text)
text

DETAIL DES PRESTATIONS (les minutes sont exprimées en centièmes d'heures) Quantité Taux Montant
HEURES TOTAL H.T TAUX T.V.A T.V.A TOTAL T.T.CTVA Intracommunautaire : FR18894239607ONE WORK                                                                    
211B AVENUE CHARLES DE                
92200      NEUILLY SUR SEINE               
SAS au capital de 100 000 euros                             
Garant Fin : GROUPAMA, 132 rue des 3 fontanot, Nanterre     Siret : 89423960700043 APE : 7820ZTel : 01 86 90 93 50      
Adresse de règlement :
 
 BG APPRO                                                                    
6 RUE DU CANAL                        
ZAC DE LA MARINIERE                   
91070      BONDOUFLE                       
FACTURE  Double Client : 010 110007 / CLI M S CTR  Q     
FACTURE  010/1000989 Date  : 31/12/2022 Page 3
Valeur en votre aimable règlement
par Virement                      
au  31/01/2023
*T.V.A Pay
S'agissant 
entraine à 
Nous nous 
               P


"DETAIL DES PRESTATIONS (les minutes sont exprimées en centièmes d'heures) Quantité Taux Montant\nHEURES TOTAL H.T TAUX T.V.A T.V.A TOTAL T.T.CTVA Intracommunautaire : FR18894239607ONE WORK                                                                    \n211B AVENUE CHARLES DE                \n92200      NEUILLY SUR SEINE               \nSAS au capital de 100 000 euros                             \nGarant Fin : GROUPAMA, 132 rue des 3 fontanot, Nanterre     Siret : 89423960700043 APE : 7820ZTel : 01 86 90 93 50      \nAdresse de règlement :\n \n BG APPRO                                                                    \n6 RUE DU CANAL                        \nZAC DE LA MARINIERE                   \n91070      BONDOUFLE                       \nFACTURE  Double Client : 010 110007 / CLI M S CTR  Q     \nFACTURE  010/1000989 Date  : 31/12/2022 Page 3\nValeur en votre aimable règlement\npar Virement                      \nau  31/01/2023\n*T.V.A Pay\nS'agissant \nentraine à \nNous nous

## Using Spacy

In [4]:
import spacy
from spacy import displacy
from spacy import matcher

In [5]:
# downloding spacy small english model
# !python -m spacy download en_core_web_sm

In [6]:
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)

In [7]:
# dispacy entity visualizer
displacy.render(doc, style='ent', jupyter=True)

In [48]:
# method extracts bill number using spacy matcher
def extract_bill_number(doc):
    # define the pattern
    pattern = [{'POS': 'PROPN'}, {'POS': 'NUM'}]

    # initialize matcher with a vocab
    Matcher = matcher(nlp.vocab)
    Matcher.add('BILL_NUMBER', None, pattern)

    # find matches in the doc
    matches = Matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        return span.text

In [49]:
extract_bill_number(text)

TypeError: 'module' object is not callable

In [39]:
# method extracts client name from text
def extract_client_name(text):
    doc = nlp(text)
    res = []
    for ent in doc.ents:
        if ent.label_ == 'ORG':
            res.append(ent.text)
    return res

In [40]:
extract_client_name(text)

['AVENUE', 'SAS', 'ZAC DE LA', 'CTR']

In [8]:
# import pytorch tensor
import torch

In [11]:
X = torch.tensor([[1,2], [3,4]])
Y = torch.tensor([[5,6], [7,8]])

In [12]:
(X@Y).sum()

tensor(134)